In [ ]:
import pandas as pd
from fastapi import FastAPI
## Evaluation
df = pd.read_csv('../Data/df_clean_final.csv')
app = FastAPI()

In [ ]:
@app.get("/api/desempeno-ventas/{tienda_id}")
def obtener_desempeno_ventas(tienda_id: int):
    df_tienda = df[df['TIENDA_ID'] == tienda_id].copy()
    df_tienda.sort_values(by='MES_ID', inplace=True)

    if df_tienda.empty:
        return {"error": "TIENDA_ID no encontrado en datos de ventas."}

    if len(df_tienda) < 2:
        return {"error": "No hay suficientes registros históricos para evaluar ventas."}

    venta_actual = float(df_tienda['VENTA_TOTAL'].iloc[-1])
    promedio_6m = float(df_tienda['VENTA_TOTAL'].iloc[-7:-1].mean() if len(df_tienda) >= 7 else df_tienda['VENTA_TOTAL'].iloc[:-1].mean())
    maximo = float(df_tienda['VENTA_TOTAL'].max())
    minimo = float(df_tienda['VENTA_TOTAL'].min())

    delta = venta_actual - promedio_6m
    porcentaje = (delta / promedio_6m) * 100 if promedio_6m != 0 else None

    return {
        "venta_ultimo_mes": venta_actual,
        "promedio_6_meses_previos": promedio_6m,
        "venta_maxima_historica": maximo,
        "venta_minima_historica": minimo,
        "comparativo_vs_promedio_pct": round(porcentaje, 2) if porcentaje is not None else None
    }

{
  "error": "TIENDA_ID no encontrado en datos de ventas."
}


In [ ]:
@app.get("/api/perfil-demografico/{tienda_id}")
def obtener_perfil_demografico(tienda_id: int):
    fila = df[df['TIENDA_ID'] == tienda_id]

    if fila.empty:
        return {"error": "TIENDA_ID no encontrado en datos demográficos."}

    fila = fila.iloc[0]

    return {
        "poblacion_total": int(fila['POBTOT']),
        "total_hogares": int(fila['TOTHOG']),
        "poblacion_economicamente_activa": int(fila['PEA']),
        "viviendas_con_automovil": int(fila['VPH_AUTOM'])
    }

{
  "poblacion_total": 2934,
  "total_hogares": 850,
  "poblacion_economicamente_activa": 1435,
  "viviendas_con_automovil": 732
}


In [ ]:
@app.get("/api/historial-ventas/{tienda_id}")
def obtener_historial_ventas(tienda_id: int):
    df_tienda = df[df['TIENDA_ID'] == tienda_id].copy()

    if df_tienda.empty:
        return {"error": "TIENDA_ID no encontrado en datos de ventas."}

    df_tienda.sort_values(by="MES_ID", inplace=True)

    historial = [
        {"mes_id": int(row["MES_ID"]), "venta_total": float(row["VENTA_TOTAL"])}
        for _, row in df_tienda.iterrows()
    ]

    return {
        "tienda_id": tienda_id,
        "historial_ventas": historial
    }